# RAG Search Example
- 建立向量数据
- 使用RAG增强

In [2]:
! pip install --quiet faiss-cpu tiktoken

In [10]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# 定义一个自定义处理逻辑
process_data = RunnableLambda(lambda x: x.upper())

# 定义一个直接传递输入的 Runnable
passthrough = RunnablePassthrough()

# 将两个 Runnables 组合成一个任务链
chain = passthrough | process_data

# 执行任务链
result = chain.invoke("hello world")
print(result)  # 输出: HELLO WORLD

HELLO WORLD


In [5]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 引入 runnables
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_ollama.chat_models import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
vectorstore = FAISS.from_texts(
  ["""CaiXukun is working in JianYu"""],
    embedding=OllamaEmbeddings(model="bge-m3")
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}
Question:{question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOllama(model="qwen2.5:7b")

chain = (
  # 定义一个字典，其中 "context" 使用 retriever 检索器，"question" 使用 RunnablePassthrough 直接传递输入
  {"context": retriever, "question": RunnablePassthrough()}
  # 将上述字典作为输入，传递给 prompt 模板，用于生成提示
  | prompt
  # 将生成的提示传递给模型（ChatOllama）进行推理
  | model
  # 将模型的输出通过 StrOutputParser 解析为字符串
  | StrOutputParser()
)

In [9]:
chain.invoke("Where did CaiXukun work")

'According to the provided context, CaiXukun works at JianYu.'

### 自定义

In [11]:
template = """Answer the question based only on the following context:
{context}
Question:{question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
  {
    # 定义输入字典，
    # 其中 "context" 是通过 retriever 检索器获取的内容，
    # retriever 的输入是从 "question" 字段中提取的值
    # itemgetter 用于从输入字典中提取指定字段的值
    "context": itemgetter("question") | retriever,
    # "question" 字段直接从输入中提取
    "question": itemgetter("question"),
    # "language" 字段直接从输入中提取，用于指定回答的语言
    "language": itemgetter("language"),
  }
  # 将上述输入字典传递给 prompt 模板，用于生成提示
  | prompt
  # 将生成的提示传递给模型（ChatOllama）进行推理
  | model
  # 将模型的输出通过 StrOutputParser 解析为字符串
  | StrOutputParser()
)

In [ ]:
chain.invoke({"question":"Where did CaiXukun work", "language":"Chinese"})

'蔡徐坤在jianyu工作。'